# Edge AI Prototype: Image Classification & TFLite Conversion

## Goal
Train a lightweight image classification model (simulating recyclable item recognition) and convert it to TensorFlow Lite for edge deployment.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

print(f"TensorFlow version: {tf.__version__}")

## 1. Load Dataset
For this prototype, we will use the **Fashion MNIST** dataset as a proxy for "recyclable items" (e.g., T-shirt/top could be 'Cloth', Sneaker could be 'Rubber', etc.). In a real scenario, you would load a custom dataset of plastic, glass, and paper.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f"Training data shape: {train_images.shape}")
print(f"Test data shape: {test_images.shape}")

## 2. Build a Lightweight Model
We use a simple Convolutional Neural Network (CNN) suitable for edge devices.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

## 3. Train the Model

In [ ]:
history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

## 4. Convert to TensorFlow Lite
We convert the trained model to the `.tflite` format. We also apply quantization to reduce model size.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optimization for size/latency
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the model
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

print(f"TFLite model saved to {tflite_model_file}")

## 5. Test TFLite Model Inference
Run inference using the TFLite interpreter to verify it works.

In [ ]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on the first image from test set
input_shape = input_details[0]['shape']
input_data = np.array(test_images[0:1], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)

print(f"Predicted Class Index: {predicted_class}")
print(f"Actual Label: {test_labels[0]}")